<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60">

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'>


Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej".   
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Statistical machine learning - Notebook 4, version for students
**Author: Michał Ciach**  
**Date: 5.11.2021**

## Description

In todays class, we will learn some aspects of parameter and interval estimation.  
In the first section, we will focus on estimating the mean value of a distribution using a statistical sample.   
In the next section, we'll calculate confidence intervals for the mean.  


In [26]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1GW1pjKOCoKOlC4Jqbqql_ghYD_n0iC6O
!gdown https://drive.google.com/uc?id=1FInZ2jrlZGNColU4sHF9JKGHP39fTVut
!gdown https://drive.google.com/uc?id=1n1qS6dcVVKcVJOuUIIm0VTz6cSyrtzDH

Downloading...
From: https://drive.google.com/uc?id=1GW1pjKOCoKOlC4Jqbqql_ghYD_n0iC6O
To: /content/BDL municipality incomes 2015-2020.csv
100% 228k/228k [00:00<00:00, 49.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FInZ2jrlZGNColU4sHF9JKGHP39fTVut
To: /content/BDL municipality area km2 2015-2020.csv
100% 180k/180k [00:00<00:00, 25.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1n1qS6dcVVKcVJOuUIIm0VTz6cSyrtzDH
To: /content/BDL municipality population 2015-2020.csv
100% 222k/222k [00:00<00:00, 80.6MB/s]


## Data & library imports

In [27]:
import pandas as pd
import numpy as np
import numpy.random as rd
import plotly.express as px
from scipy.stats import norm

In [28]:
income = pd.read_csv('BDL municipality incomes 2015-2020.csv', sep=';', dtype={'Code': 'str'})
area = pd.read_csv('BDL municipality area km2 2015-2020.csv', sep='\t', dtype={'Code': 'str'})
population = pd.read_csv('BDL municipality population 2015-2020.csv', sep='\t', dtype={'Code': 'str'})

In [29]:
income

,Code,Region,2015,2016,2017,2018,2019,2020
0,0201011,Bolesławiec (1),9.776646e+07,9.658595e+07,1.003549e+08,1.000265e+08,1.107985e+08,1.138563e+08
1,0201022,Bolesławiec (2),3.107224e+07,2.913815e+07,3.683091e+07,3.484836e+07,3.871533e+07,4.288890e+07
2,0201032,Gromadka (2),1.089941e+07,1.313974e+07,1.454154e+07,2.705794e+07,2.572157e+07,2.754443e+07
3,0201043,Nowogrodziec (3),1.856915e+07,2.941747e+07,3.188345e+07,3.178886e+07,3.913420e+07,3.341908e+07
4,0201052,Osiecznica (2),1.674647e+07,1.709802e+07,1.760182e+07,1.984173e+07,2.177626e+07,2.484304e+07
...,...,...,...,...,...,...,...,...
2504,3218043,Resko (3),1.523513e+07,1.448583e+07,1.611091e+07,1.763241e+07,1.760038e+07,1.941628e+07
2505,3218053,Węgorzyno (3),1.069708e+07,1.037428e+07,1.082646e+07,1.201310e+07,1.359405e+07,2.503944e+07
2506,3261011,Koszalin (1),2.697281e+08,2.891235e+08,3.054048e+08,3.245613e+08,3.432316e+08,3.840694e+08
2507,3262011,Szczecin (1),1.350327e+09,1.372046e+09,1.343422e+09,1.431826e+09,1.545381e+09,1.739014e+09


## Point estimation

In the first notebook, we used the `pd.describe()` function that showed us, among others, the mean and the median income of municipalities calculated using all the observations.  
One of the main strengths of statistical theory is that it allows us to estimate the mean income using a sample of randomly selected municipalities, and, most importantly, to estimate the uncertainty of our estimation.  
This is the main reason why we derive properties of estimators, such as their expected value and variance.  

**Exercise 1.** Select the data on the municipality incomes from the year 2020 and remove the missing observations. Calculate the base-10 logarithms of the incomes. We'll call the data obtained this way the *log-income* of municipalities.  
Calculate the mean and standard deviation of the log-income. We'll call them the *true* parameters, as opposed to estimators computed from a sample.  

Using the equations for the expected value and the standard deviation of the estimator of the mean, $\hat{\mu}_X = \sum_{i=1}^N X_i/N$ where $X_i$ is the log-income of a randomly selected municipality, calculate the number of randomly selected municipalities $N$ that we need to take so that the standard deviation of the estimator is equal to 1\% of the true mean of the log-income.  

Select a sample of the size $N$ and calculate the mean of the log-incomes. Compare it to the true mean log-income.  

Repeat the sampling 1000 times to obtain 1000 values of the estimator of the mean log-income. Use them to calculate the standard deviation of the estimator of the mean and verify that it's close to 1\% of the true mean. Next, calculate the mean (i.e. estimate the expected value) of the estimator of the mean. Can you see any bias? Does it agree with the theoretical result about the bias of $\hat{\mu}$? 

Plot the 1000 values of the estimator of the mean on a histogram. Annotate the histogram with the true mean (any way you want, I typically use a red dot at the bottom of the histogram or a vertical line).  
Is the distribution of the estimator similar to the distribution of the log-income, or is it similar to some other distribution?  

Can we use the estimated average log-income to estimate the average income of municipalities?  


In [30]:
income_trans = income['2020']
income_trans = income_trans.dropna()

income_log = np.log10(income_trans)

In [31]:
income_log_mean = income_log.mean()
income_log_std = income_log.std()

print(income_log_mean)
print(income_log_std)

7.290919587043024
0.4380297867209015


In [32]:
N = income_log_std ** 2 / (0.01 * income_log_mean) ** 2
N = int(np.ceil(N))

print(N)

37


In [33]:
idx = np.random.choice(income_log.shape[0], N, replace=False)
income_example = income_log.iloc[idx]

print(income_example.mean())

7.346500408645508


In [34]:
means = []
for i in range(1000):
    idx = np.random.choice(income_log.shape[0], N, replace=False)
    mean = income_log.iloc[idx].mean()
    means.append(mean)

means = np.array(means)

In [35]:
means_mean = np.mean(means)
means_std = np.std(means)

print(means_mean * 0.01)
print(means_std)

0.07292025688769066
0.07134321154469715


In [36]:
plot = px.histogram(pd.DataFrame({'mean': means}), x='mean')
plot.add_shape(type='line', x0=income_log_mean, y0=0, x1=income_log_mean, y1=60,
               line_color='yellow')

**Exercise 2.** In the previous exercise we were dealing with estimating the mean log-income of a municipality and we have analyzed the properties of the estimator $\hat{\mu}_X = \sum_{i=1}^N \log_{10} (I_i) / N$, where $I_i = 10^{X_i}$ is the income of the $i$-th randomly selected municipality. The estimator of the mean income is given by the equation $\hat{\mu}_I = \sum_{i=1}^N I_i / N$.

Consider a statistic given by the equation $\hat{\zeta} = 10^{\hat{\mu_X}}$. Is it an estimator of the average income? Does it correspond to some well-known mathematical object, e.g. some kind of a mean value? 

Regardless of the answer, let's use $\hat{\zeta}$ to estimate the average income. Use the randomly sampled values of $\hat{\mu_X}$ from the previous exercise to estimate the expected value and the standard deviation of $\hat{\zeta}$. Based on the results, do you expect that $\hat{\zeta}$ is a biased estimator of the average income? Does $\hat{\zeta}$ have a lower or a higher variance than $\hat{\mu}_I$?  

Try to confirm your expectations by deriving formulas for the expected value of $\hat{\zeta}$. If it is biased, then how does the bias scale with the number of observations? Check this either theoretically by analyzing equations or empirically by varying the sample size.  

Plot the values of the estimators $\hat{\zeta}$ and $\hat{\mu}_I$ on two boxplots side-by-side and annotate it with the true average income (for example, using a horizontal line).  
Which estimator seems better?  

Calculate the mean squared errors of the two estimators and compare them.   

What would be the sample size required so that the standard deviation of $\hat{\mu}_I$ is equal to 1\% of the average municipality income? 

In [37]:
income_mean = income_trans.mean()
income_std = income_trans.std()

print(income_mean)
print(income_std)

49129101.345155515
300858589.9313959


In [38]:
zeta_mean = np.mean(10**means)
zeta_std = np.std(10**means)

print(zeta_mean)
print(zeta_std)

19857101.000000607
3306890.801792729


In [39]:
mu_means = []
for i in range(1000):
    idx = np.random.choice(income_trans.shape[0], N, replace=False)
    mean = income_trans.iloc[idx].mean()
    mu_means.append(mean)

mu_means = np.array(mu_means)

In [40]:
mu_mean = mu_means.mean()
mu_std = mu_means.std()

print(mu_mean)
print(mu_std)

49367631.42021514
51084674.92049088


In [41]:
plot = px.box(pd.melt(pd.DataFrame({'mu': mu_means, 'zeta': 10**means})), x='variable', y='value')
plot.add_shape(type='line', x0=-0.5, y0=income_mean, x1=1.5, y1=income_mean,
               line_color='yellow')

In [42]:
def MSE(true, pred):
    return np.mean((true-pred)**2)

print(MSE(income_trans, np.mean(mu_means)))
print(MSE(income_trans, np.mean(10**means)))

9.04794054842097e+16
9.133619859182002e+16


In [43]:
good_N = income_std ** 2 / (0.01 * income_mean) ** 2
good_N = int(np.ceil(good_N))

print(good_N)

375014


## Interval estimation

In the previous section, we've learned how to quantify and analyze the uncertainty of an estimation by analyzing the standard deviation of the estimator.  
In this section, we will learn a different technique - the estimation of *confidence intervals*, i.e. intervals which are likely to contain the true value of the parameter of interest.   
We say that a confidence interval has a confidence level $\alpha$ if it contains the true value of the parameter with probability $\alpha$ (note: some authors use a different terminology and would call this a level $1-\alpha$).  

We will use so-called *assymptotic intervals for the mean*, given by the equation 

$$\left (\hat{\mu} - \frac{q_{(1+\alpha)/2}\hat{\sigma}}{\sqrt{N}},\quad \hat{\mu} + \frac{q_{(1+\alpha)/2}\hat{\sigma}}{\sqrt{N}} \right ), $$
where $q_{(1+\alpha)/2}$ is the quantile of the standard normal distribution at the level of $(1+\alpha)/2$.  

Note: you may have heard elsewhere that there are certain theoretic and philosophical problems with the phrase "a confidence interval contains the true parameter value with a given probability". We'll ignore them in this class.  

**Exercise 3.** Select the data about the **population** of municipalities in 2020, remove missing values and log10-transform them.  
Calculate the true mean and standard deviation of the population.  
Select a random sample of size $N$, calculate its average, and construct a confidence interval for the mean on the confidence level of 95%.  
See if the true value is inside the interval.  

Plot the data on a histogram and annotate it with the true mean, the estimated mean, and the confidence interval.

Repeat the sampling and calculation of the confidence interval 1000 times, check the proportion of samples for which the confidence interval contains the true mean. Is it close to 95%? Why/why not?   

What would be the sample size required for the confidence interval to have length equal to 10% of the true mean, if we used the true mean and standard deviation instead of their estimators in the definition of the confidence interval?  

Select a single confidence interval.  
Repeat the selection of 1000 samples. For each sample, calculate an empirical mean and check if it's within the interval.  
What's the proportion of means that are within the interval? Is it close to 95%? Why/why not?  

In [44]:
population_trans = population['2020']
population_trans = population_trans.dropna()

population_log = np.log10(population_trans)

In [45]:
population_log_mean = population_log.mean()
population_log_std = population_log.std()

print(population_log_mean)
print(population_log_std)

3.8740524041207163
0.3679144547533364


In [46]:
from scipy.stats import norm

N = 38
q = 0.05
ppf = norm.ppf(q/2)

def get_interval():
    idx = np.random.choice(population_log.shape[0], N, replace=False)
    population_sample = population_log.iloc[idx]
    population_sample_mean = population_sample.mean()
    population_sample_std = population_sample.std()
    
    interval_value = np.abs(ppf * population_sample_std / np.sqrt(N))
    lower_95 = population_sample_mean - interval_value
    upper_95 = population_sample_mean + interval_value

    return lower_95, upper_95, population_sample_mean

In [47]:
lower_95, upper_95, population_sample_mean = get_interval()

print(population_sample_mean)
print(lower_95)
print(upper_95)

3.849990624254747
3.7311833626190856
3.9687978858904085


In [48]:
plot = px.histogram(pd.DataFrame({'population_log': population_log}), x='population_log')

plot.add_shape(type='line', x0=lower_95, y0=0, x1=lower_95, y1=270,
               line_color='yellow', name='lower 95')

plot.add_shape(type='line', x0=upper_95, y0=0, x1=upper_95, y1=270,
               line_color='yellow', name='upper 95')

plot.add_shape(type='line', x0=population_sample_mean, y0=0, x1=population_sample_mean, y1=270,
               line_color='violet', name='mean of sample of population')

plot.add_shape(type='line', x0=population_log_mean, y0=0, x1=population_log_mean, y1=270,
               line_color='red', name='population mean')

In [49]:
cntr = 0
for _ in range(1000):
    lower_95, upper_95, _ = get_interval()
    if population_log_mean >= lower_95 and population_log_mean < upper_95:
          cntr += 1

print(cntr)

940


In [50]:
population_log_N = (2 * ppf * population_log_std) ** 2 / (0.1 * population_log_mean) ** 2
population_log_N = int(np.ceil(population_log_N))

print(population_log_N)

14


In [51]:
lower_95, upper_95, _ = get_interval()

cntr = 0
for _ in range(1000):
    idx = np.random.choice(population_log.shape[0], N, replace=False)
    population_sample = population_log.iloc[idx]
    population_sample_mean = population_sample.mean()

    if population_sample_mean >= lower_95 and population_sample_mean < upper_95:
        cntr += 1

print(cntr/1000)

0.929


**Exercise 4 - homework.** Repeat the previous exercise on the data without the log-transformation.  
Explain in what way the confidence intervals work worse for the non-transformed data set than for the log10-population data, and why this happens.   
Additionally, plot the distribution of the lengths and the left boundaries of the confidence intervals.

In [52]:
population_mean = population_trans.mean()
population_std = population_trans.std()

print(population_mean)
print(population_std)

12989.054776395871
42314.9873346266


In [53]:
N = 38
q = 0.05
ppf = norm.ppf(q/2)
    
def get_interval():
    idx = np.random.choice(population_trans.shape[0], N, replace=False)
    population_sample = population_trans.iloc[idx]
    population_sample_mean = population_sample.mean()
    population_sample_std = population_sample.std()

    interval_value = np.abs(ppf * population_sample_std / np.sqrt(N))
    lower_95 = population_sample_mean - interval_value
    upper_95 = population_sample_mean + interval_value

    return lower_95, upper_95, population_sample_mean

In [54]:
lower_95, upper_95, population_sample_mean = get_interval()

print(population_sample_mean)
print(lower_95)
print(upper_95)

27076.526315789473
-402.23203426654436
54555.284665845495


In [55]:
plot = px.histogram(pd.DataFrame({'population_trans': population_trans}), x='population_trans')

plot.add_shape(type='line', x0=lower_95, y0=0, x1=lower_95, y1=1400,
               line_color='yellow', name='lower 95')

plot.add_shape(type='line', x0=upper_95, y0=0, x1=upper_95, y1=1400,
               line_color='yellow', name='upper 95')

plot.add_shape(type='line', x0=population_sample_mean, y0=0, x1=population_sample_mean, y1=1400,
               line_color='violet', name='mean of sample of population')

plot.add_shape(type='line', x0=population_mean, y0=0, x1=population_mean, y1=1400,
               line_color='red', name='population mean')

In [56]:
cntr = 0
lengths = []
left_boundaries = []

for _ in range(1000):
    lower_95, upper_95, _ = get_interval()
    lengths.append(upper_95-lower_95)
    left_boundaries.append(lower_95)

    if population_mean >= lower_95 and population_mean < upper_95:
          cntr += 1

print(cntr)

667


In [57]:
population_N = (2 * ppf * population_std) ** 2 / (0.1 * population_mean) ** 2
population_N = int(np.ceil(population_N))
population_N 

16308

In [58]:
lower_95, upper_95, _ = get_interval()

cntr = 0
for _ in range(1000):
    idx = np.random.choice(population_trans.shape[0], N, replace=False)
    population_sample = population_trans.iloc[idx]
    ps_mean = population_sample.mean()

    if ps_mean >= lower_95 and ps_mean < upper_95:
        cntr += 1

print(cntr/1000)

0.685


In [59]:
px.histogram(pd.DataFrame({'lengths': lengths}), x='lengths')

In [60]:
px.histogram(pd.DataFrame({'left boundaries': left_boundaries}), x='left boundaries')

<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60">

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'>


Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej".   
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>